# Ollama Embeddings
[Ollama](https://ollama.com/download) can also serve embedding models locally. Before executing the following code, you need to run `ollama pull embedinggemma` once to download the embedding model. Also, depending on how you installed ollama, you may have to execute it in a terminal window using this command, before executing this notebook:

```
ollama serve
```

As you will see, we access the local embedding models offered via ollama using the OpenAI API as shown in previous examples. We just exchange the `base_url` and we do not need to provide an API-Key.

In [6]:
import openai
openai.__version__

'1.41.0'

## Creating Embeddings with Ollama
We define a helper function to generate text embeddings using the local Ollama endpoint. The function connects to the local Ollama server and uses the "embedinggemma" model to create vector representations of text.

In [7]:
def embed_ollama(text, model="embeddinggemma"):
    """A helper function that generates embeddings using ollama and returns the embedding vector."""
    
    # setup connection to the local Ollama server
    client = openai.OpenAI()
    client.base_url = "http://localhost:11434/v1"
    client.api_key = "none"  # No API key needed for local ollama
    
    # create embedding
    response = client.embeddings.create(
        input=text,
        model=model
    )
    
    # extract embedding vector
    return response.data[0].embedding

Let's test the embedding function with a simple example:

In [8]:
# Test with a simple text
test_text = "Hello, this is a test sentence for embeddings."
embedding = embed_ollama(test_text)

print(f"Text: {test_text}")
print(f"Embedding dimension: {len(embedding)}")
print(f"First 5 values: {embedding[:5]}")

Text: Hello, this is a test sentence for embeddings.
Embedding dimension: 768
First 5 values: [-0.16048418, -0.002961286, 0.014041578, -0.029707532, -0.009763586]


## Working with Multiple Texts
Let's generate embeddings for multiple texts and compare them:

In [9]:
# Define some sample texts
texts = [
    "The cat sat on the mat.",
    "A feline rested on the carpet.",
    "The dog ran in the park.",
    "Machine learning is fascinating.",
    "Artificial intelligence transforms technology."
]

# Generate embeddings for all texts
embeddings = {}
for i, text in enumerate(texts):
    embeddings[f"text_{i+1}"] = embed_ollama(text)
    print(f"Generated embedding for text {i+1}: {text[:30]}...")

print(f"\nGenerated {len(embeddings)} embeddings successfully!")

Generated embedding for text 1: The cat sat on the mat....
Generated embedding for text 2: A feline rested on the carpet....
Generated embedding for text 2: A feline rested on the carpet....
Generated embedding for text 3: The dog ran in the park....
Generated embedding for text 3: The dog ran in the park....
Generated embedding for text 4: Machine learning is fascinatin...
Generated embedding for text 4: Machine learning is fascinatin...
Generated embedding for text 5: Artificial intelligence transf...

Generated 5 embeddings successfully!
Generated embedding for text 5: Artificial intelligence transf...

Generated 5 embeddings successfully!


## Visualizing Embeddings
Let's use PCA to reduce the dimensionality and visualize the embeddings in 2D space:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA

# Apply PCA to reduce to 2 dimensions
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(list(embeddings.values()))

# Create scatter plot
plt.figure(figsize=(10, 8))
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], s=100, alpha=0.7)

# Add text labels for each point
for i, text in enumerate(texts):
    plt.annotate(f"{i+1}: {text[:25]}...", 
                (embeddings_2d[i, 0], embeddings_2d[i, 1]),
                xytext=(5, 5), textcoords='offset points',
                fontsize=9, alpha=0.8)

plt.title('Text Embeddings Visualization (PCA Projection)', fontsize=14)
plt.xlabel(f'Principal Component 1 (explained variance: {pca.explained_variance_ratio_[0]:.3f})')
plt.ylabel(f'Principal Component 2 (explained variance: {pca.explained_variance_ratio_[1]:.3f})')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Total explained variance: {sum(pca.explained_variance_ratio_):.3f}")

TypeError: float() argument must be a string or a real number, not 'dict_values'

## Semantic Search Example
We can use embeddings for semantic search - finding the most similar text to a query:

In [ ]:
def semantic_search(query, texts, top_k=3):
    """Find the most similar texts to a query using embeddings."""
    
    # Get embedding for the query
    query_embedding = embed_ollama(query)
    
    # Get embeddings for all texts
    text_embeddings = [embed_ollama(text) for text in texts]
    
    # Calculate similarities
    similarities = []
    for text_emb in text_embeddings:
        # Cosine similarity between query and text embeddings
        similarity = np.dot(query_embedding, text_emb) / (
            np.linalg.norm(query_embedding) * np.linalg.norm(text_emb)
        )
        similarities.append(similarity)
    
    # Get top-k most similar texts
    indexed_similarities = [(i, sim) for i, sim in enumerate(similarities)]
    indexed_similarities.sort(key=lambda x: x[1], reverse=True)
    
    return indexed_similarities[:top_k]

# Example search
query = "animal sitting down"
results = semantic_search(query, texts)

print(f"Query: '{query}'")
print("\nMost similar texts:")
for rank, (idx, similarity) in enumerate(results, 1):
    print(f"{rank}. Text {idx+1} (similarity: {similarity:.3f}): {texts[idx]}")

## Exercise
1. Try different texts and see how the embeddings cluster in the visualization
2. Experiment with different queries in the semantic search function
3. Explore other embedding models available in Ollama by running `ollama list` in your terminal
4. Compare the results with different embedding models (if you have others installed)


In [ ]:
# Your experiments here